In [1]:
import csv
import json
from typing import TypedDict
from collections import defaultdict

In [18]:
file = "../../data/toflit18_all_flows.csv"
product_classification = "product_sitc_simplEN"

In [3]:
class DataPoint(TypedDict):
    product: str 
    import_dk: int
    import_dkb: int
    export_dk: int
    export_dkb: int

In [64]:
report_dk = defaultdict(dict)
report_others = defaultdict(dict)
import_dk_by_product_by_partner = defaultdict(dict)
def import_partner_class(partner):
    if partner in ["Asie", "Afrique", "Amériques"]:
        return "colonies"
    if partner == "France":
        print(partner)
        return "France"
    return 'Monde'

regions = set()
partners = set()
with open(file, "r") as muerte:
    reader = csv.DictReader(muerte)
    for i, row in enumerate(reader):
        if row["year"] == "1789":
            product = row[product_classification]
            region = row["customs_region_grouping"]

            value = float(row["value"]) if row["value"] else 0.0
            # process mirrors flows : France imports from Dunkerque 
            if row["export_import"] == "Imports" and row["partner_simplification"] == "Dunkerque":
                regions.add(region)
                if product:
                    if region not in report_others[product]:
                        report_others[product][region] = {}
                    report_others[product][region]["product"] = product
                    report_others[product][region]["region"] = region
                    report_others[product][region]["import"] = report_others[product][region].get("import", 0) + value
            # process mirrors flows : France exports to Dunkerque
            if row["export_import"] == "Exports" and row["partner_simplification"] == "Dunkerque":
                if product:
                    import_dk_by_product_by_partner[product]["France"] = import_dk_by_product_by_partner[product].get("France", 0 ) + value

            if row["customs_office"] in ['Dunkerque', 'Dunkerque-Basseville']:
                if product:
                    report_dk[product]["product"] = product
                    if row["customs_office"] == 'Dunkerque':
                        if row["export_import"] == "Exports":
                            report_dk[product]["export_dk"] = report_dk[product].get("export_dk", 0) + value
                        elif row["export_import"] == "Imports":
                            report_dk[product]["import_dk"] = report_dk[product].get("import_dk", 0) + value
                            partners.add(row['partner_grouping'])
                            partner = import_partner_class(row["partner_grouping"])
                            import_dk_by_product_by_partner[product][partner] = import_dk_by_product_by_partner[product].get(partner,0) + value
                    elif row["customs_office"] == 'Dunkerque-Basseville':
                        if row["export_import"] == "Exports":
                            report_dk[product]["export_dkb"] = report_dk[product].get("export_dkb", 0) + value
                        elif row["export_import"] == "Imports":
                            report_dk[product]["import_dkb"] = report_dk[product].get("import_dkb", 0) + value

In [ ]:
import_dk_by_product_by_partner
# partners

In [73]:
all_data = []
all_exp_to_fr = 0
products = report_dk.keys() | report_others.keys()

for product in products:
    flows = report_dk[product] if product in report_dk else {}
    export_to_fr = sum([dp['import'] for (region,dp) in report_others.get(product,{}).items()])
    all_exp_to_fr += export_to_fr
    export_to_colonies = flows.get("export_dk", 0)
    total = flows.get("import_dk",0) + import_dk_by_product_by_partner[product].get("France",0)

    all_data.append({"value":total - export_to_fr - export_to_colonies, "product": product, "partner" : "entrepôt (estimation)", "group": "export"})
    all_data.append({"value":export_to_fr, "product": product, "partner" : "France", "group": "export"})
    all_data.append({"value":export_to_colonies, "product": product, "partner" : "colonies ou re-exp", "group": "export"})

    if product in import_dk_by_product_by_partner:
        for (partner, value) in import_dk_by_product_by_partner[product].items():
            all_data.append({"value":value, "product": product, "partner" : partner, "group": "import"})


In [ ]:
all_data

In [ ]:
with open("data/report_dunkerque.json", "w") as f:
    json.dump(report_dk, f)

In [ ]:
with open("data/report_other_regions.json", "w") as f:
    json.dump(report_others, f)

In [75]:
with open("data/export_dunkerque_entreprôt.json", "w") as f:
    json.dump(all_data, f)